# Neural networks

**Universal approximation theorem: Neural Networks can approximate any continuous function.**

## Outline:

- A visual [demonstration](http://neuralnetworksanddeeplearning.com/chap4.html) that neural networks (NNs) can compute any function.
- Like any Machine Learning (ML) algorithm, training a neural network requires minimizing some loss function (defined as the distance between the predicted value and target value).
- The minimization is done using an algorithm called gradient descent, or a variation called stochastic/minibach gradient descent. 

## Using gradient descent on linear regression

The simplest machine learning algorithm is [linear regression](https://en.wikipedia.org/wiki/Linear_regression). We will code up linear regression from scratch with a twist: we will use gradient descent, which is also how neural networks learn. Most of this lesson is pretty much stolen from Jeremy Howard's fast.ai [lesson zero](https://www.youtube.com/watch?v=ACU-T9L4_lI)

- In linear regression, we assume that $y = w_0 + w_1 x $, where $y$ is the expected output.
- We look for the $w$ coefficients that give the 'best' prediction ($\tilde{y}$) for the output. The best prediction is defined by minimizing some cost function. For linear regression, we generally minimize the mean sequare error between the expected output $y$ and the prediction ($\tilde{y}$).

### Linear regression from scratch

We will learn the parameters $w_0$ and $\mathbf{w}_1$ of a line.

In [ ]:
# Import plotting packages
from IPython.display import Image, HTML
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt

import numpy as np

# Import machine-learning packages
import torch
from torch import nn

%matplotlib inline

Set random seed for reproducibility


In [ ]:
np.random.seed(42)

<center>
<img src='https://miro.medium.com/v2/resize:fit:1032/1*WswH2fPx0bf_JFRMm8V-HA.gif' width=500/>
</center>

Image is taken from [here](https://blog.insightdatascience.com/a-quick-introduction-to-vanilla-neural-networks-b0998c6216a1).

### What is [PyTorch](https://pytorch.org/)?

PyTorch is a Python-based scientific computing package targeted at two sets of audiences:

- A replacement for NumPy to use the power of GPUs
- A deep learning research platform that provides flexibility and speed

### What is a [Tensor](https://pytorch.org/tutorials/beginner/basics/tensorqs_tutorial.html)?
Tensor is a data structure which is a fundamental building block of PyTorch. Tensors are pretty much like numpy arrays, except that unlike numpy, tensors are designed to take advantage of parallel computation capabilities of a GPU
and more importantly for us - they can keep track of its gradients.

Further reading [here](https://blog.paperspace.com/pytorch-101-understanding-graphs-and-automatic-differentiation/).

#### Chose the true parameters we want to learn.

In [ ]:
w = torch.as_tensor([3.0, 2])
w

## Linear regression

### Data
Create some data points x and y which lie on the line



Documentation for:
- [ones()](https://pytorch.org/docs/stable/generated/torch.ones.html)
- [uniform_()](https://pytorch.org/docs/stable/generated/torch.Tensor.uniform_.html)

In [ ]:
n = 100
x = torch.ones(n, 2)
# Underscore functions in pytorch means replace the value (update)
x[:, 1].uniform_(-1.0, 1)

x[:5]

In [ ]:
y = x @ w + torch.rand(n)  # @ is a matrix product (similar to matmul)

In [ ]:
plt.figure(dpi=150)
plt.scatter(x[:, 1], y)
plt.xlabel("x")
plt.ylabel("y")
plt.show();

In [ ]:
w_real = torch.as_tensor([-3.0, -5])

### Loss function
If we could find a way to fit our guess for the coefficients the weights ($w_0$ and $\mathbf{w}_1$), we could use the exact same method for very complicated tasks (as in image recognition). 

We define our loss function $L$ as Mean Square Error loss as:
\begin{equation*}
L_{MSE} = \frac{1}{n} \cdot \sum_{i=1}^{n} (y_i - \tilde y_i)^2
\end{equation*}

In [ ]:
def mse(y_true, y_pred):
    return ((y_true - y_pred) ** 2).mean()

Written in terms of $w_0$ and $w_1$, our **loss function** $L$ is:

\begin{equation*}
L_{MSE} = \frac{1}{n} \cdot \sum_{i=1}^{n} (y_i - (w_0 + \mathbf{w_1} \cdot x_i))^2
\end{equation*}

In [ ]:
y_hat = x @ w_real
# Initial mean-squared error
mse(y_hat, y)

In [ ]:
plt.figure(dpi=150)
plt.scatter(x[:, 1], y, label="y")
plt.scatter(x[:, 1], y_hat, label="$\\tilde{y}$")
plt.xlabel("$x$")
plt.legend(fontsize=7);

In [ ]:
w = nn.Parameter(w_real)
w

## Gradient descent


So far, we have specified the *model* (linear regression) and the *evaluation criteria* (or *loss function*). Now we need to handle *optimization*; that is, how do we find the best values for weights ($w_0$, $w_1$) such that they best fit the linear regression line?

To know how to change $w_0$ and $w_1$ to reduce the loss, we compute the derivatives (or gradients:

\begin{equation*}
\frac{\partial L_{MSE}}{\partial w_0} = \frac{1}{n}\sum_{i=1}^{n} -2\cdot [y_i - (w_0 + \mathbf{w_1}\cdot x_i)]
\end{equation*}

\begin{equation*}
\frac{\partial L_{MSE}}{\partial \mathbf{w_1}} = \frac{1}{n}\sum_{i=1}^{n} -2\cdot [y_i - (w_0 + \mathbf{w_1}\cdot x_i)] \cdot x_i
\end{equation*}

Since we know that we can iteratively take little steps down along the gradient to reduce the loss, aka, *gradient descent*, the size of the step is determined by the learning rate ($\eta$):

\begin{equation*}
w_0^{new} = w_0^{current} -   \eta \cdot \frac{\partial L_{MSE}}{\partial w_0}
\end{equation*}

\begin{equation*}
\mathbf{w_1^{new}} = \mathbf{w_1^{current}} -  \eta \cdot \frac{\partial L_{MSE}}{\partial \mathbf{w_1}}
\end{equation*}

In [ ]:
def step(lr):
    y_hat = x @ w
    loss = mse(y, y_hat)
    # calculate the gradient of a tensor! It is now stored at w.grad
    loss.backward()

    # To prevent tracking history and using memory
    # (code block where we don't need to track the gradients but only modify the values of tensors)
    with torch.no_grad():
        # lr is the learning rate. Good learning rate is a key part of Neural Networks.
        w.sub_(lr * w.grad)
        # We want to zero the gradient before we are re-evaluate it.
        w.grad.zero_()

    return loss

In PyTorch, we need to set the gradients to zero before starting to do back propragation because PyTorch accumulates the gradients on subsequent backward passes. This is convenient while training Recurrent Neural Networks (RNNs). So, the default action is to accumulate or sum the gradients on every `loss.backward()` call.
Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly. Else the gradient would point in some other direction than the intended direction towards the minimum (or maximum, in case of maximization objectives).

Explanations about how PyTorch calculates the gradients can be found [here](https://blog.paperspace.com/pytorch-101-understanding-graphs-and-automatic-differentiation/).

In [ ]:
def lin(w1, w0, x):
    return w1 * x + w0

In [ ]:
w = torch.as_tensor([-2.0, -3])
w = nn.Parameter(w)
lr = 0.1
losses = []
epoch = 100

### Visualization

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))
ax.scatter(x[:, 0], y)
(line,) = ax.plot(
    x[:, 0],
    lin(w.detach().numpy()[0], w.detach().numpy()[1], x.detach().numpy()[:, 0]),
    c="firebrick",
)
# line, = ax.plot(x[:,0], y, c='firebrick')
ax.set_title("Loss = 0.00")
plt.close()

# Train model and perform gradient descent
for _ in range(epoch):
    loss = step(lr)
    losses.append(loss.detach().numpy())


def animate(i):
    ax.set_title("Loss = %.2f" % losses[i])
    line.set_data(
        x.detach().numpy()[:, 0],
        lin(w.detach().numpy()[0], w.detach().numpy()[1], x.detach().numpy()[:, 0]),
    )
    return (line,)


animation = FuncAnimation(fig, animate, frames=70, interval=150, blit=True);

In case you have some difficulties running the cell below without importing certain packages. Run the following code in a terminal in `L96M2lines` environment.
```shell
conda install -c conda-forge ffmpeg
```

In [ ]:
HTML(animation.to_html5_video())

In [ ]:
Image(filename="figs/Gradient_descent2.png", width=700)

In [ ]:
plt.figure(dpi=150)
plt.plot(np.array(losses))
plt.xlabel("Iteration")
plt.ylabel("Loss")
plt.title("Loss vs Iteration")
plt.show();

## Further reading
In Deep learning, we use a variation of gradient descent called [mini-batch gradient descent](https://machinelearningmastery.com/gentle-introduction-mini-batch-gradient-descent-configure-batch-size/). Instead of calculating the gradient over the whole training data before changing model weights (coefficients), we take a subset (batch) of our data, and change the values of the weights after we calculated the gradient over this subset. 